In [1]:
# STEP 1: Import the necessary modules.
import os
import cv2
import mediapipe as mp
from mediapipe.python.solutions.drawing_utils import draw_landmarks
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.tasks.python.vision import RunningMode

/Users/bennobrunner/projects/playground/fp-team2-py/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# STEP 2: Setup HandLandmarker configuration
hand_base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
face_base_options = python.BaseOptions(model_asset_path='face_landmarker.task')
hand_options = vision.HandLandmarkerOptions(base_options=hand_base_options,
                                            num_hands=2, running_mode=RunningMode.VIDEO)

face_options = vision.FaceLandmarkerOptions(base_options=face_base_options,
                                          num_faces=1, running_mode=RunningMode.VIDEO)

In [3]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        # Draw the hand landmarks.
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            solutions.hands.HAND_CONNECTIONS,
            solutions.drawing_styles.get_default_hand_landmarks_style(),
            solutions.drawing_styles.get_default_hand_connections_style())

        # Get the top left corner of the detected hand's bounding box.
        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        # Draw handedness (left or right hand) on the image.
        cv2.putText(annotated_image, f"{handedness[0].category_name}",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image

In [4]:
# STEP 4: Generate landmarks for each frame of every video
for root, _, files in os.walk("../food/data/"):
    for filename in files:  # loop through files in the current directory
        capture = cv2.VideoCapture(os.path.join(root, filename))
        hand_detector = vision.HandLandmarker.create_from_options(hand_options)
        face_datector = vision.FaceLandmarker.create_from_options(face_options)
        frameNr = 0
        while True:
            ret, frame = capture.read()
            if not ret:
                break
            # Convert the frame to a MediaPipe Image.
            image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            # Detect hand landmarks in the current frame.
            hand_detection_result = hand_detector.detect_for_video(image, frameNr)
            face_detection_result = face_datector.detect_for_video(image, frameNr)
            hand_csv = []

            # Collect landmarks and handedness for each detected hand.
            for idx in range(len(hand_detection_result.hand_landmarks)):
                handedness = hand_detection_result.handedness[idx][0].category_name
                landmarks = hand_detection_result.hand_landmarks[idx]
                hand_csv.append("\n".join(map(lambda x: f"{handedness}, x:{x.x}, y:{x.y}, z:{x.z}", landmarks)))

            for idx in range(len(face_detection_result.face_landmarks)):
                landmarks = face_detection_result.face_landmarks[idx]
                hand_csv.append("\n".join(map(lambda x: f"face, x:{x.x}, y:{x.y}, z:{x.z}", landmarks)))

            # Write the landmarks to a CSV file.
            try:
                os.mkdir(os.path.join(root, "landmarks"))
            except FileExistsError:
                pass
            with open(os.path.join(root, f"landmarks/{filename}_landmarks.csv"), "a") as f:
                f.write(f"frame {frameNr} \n")
                f.write("\n".join(hand_csv))
                f.write("\n")
            # uncomment to print landmarks into frames for examples
            # if len(hand_detection_result.hand_landmarks) != 0:
            #     annotated_image = draw_landmarks_on_image(image.numpy_view(), hand_detection_result)
            #     cv2.imwrite(os.path.join(root, f"landmarks/{filename}_frame{frameNr}.jpg"), annotated_image)

            frameNr = frameNr + 1
            print(f"Processed {filename} frame {frameNr}")
        capture.release()

OpenCV: Couldn't read video stream from file "../food/data/.DS_Store"
I0000 00:00:1754996776.676915 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1754996776.704588 60654770 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754996776.718287 60654778 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1754996776.719801 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1754996776.721408 60654391 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1754996776.728179 60654782 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. 

Processed apfel_test_1.mp4 frame 1
Processed apfel_test_1.mp4 frame 2
Processed apfel_test_1.mp4 frame 3
Processed apfel_test_1.mp4 frame 4
Processed apfel_test_1.mp4 frame 5
Processed apfel_test_1.mp4 frame 6
Processed apfel_test_1.mp4 frame 7
Processed apfel_test_1.mp4 frame 8
Processed apfel_test_1.mp4 frame 9
Processed apfel_test_1.mp4 frame 10
Processed apfel_test_1.mp4 frame 11
Processed apfel_test_1.mp4 frame 12
Processed apfel_test_1.mp4 frame 13
Processed apfel_test_1.mp4 frame 14
Processed apfel_test_1.mp4 frame 15
Processed apfel_test_1.mp4 frame 16
Processed apfel_test_1.mp4 frame 17
Processed apfel_test_1.mp4 frame 18
Processed apfel_test_1.mp4 frame 19
Processed apfel_test_1.mp4 frame 20
Processed apfel_test_1.mp4 frame 21
Processed apfel_test_1.mp4 frame 22
Processed apfel_test_1.mp4 frame 23
Processed apfel_test_1.mp4 frame 24
Processed apfel_test_1.mp4 frame 25
Processed apfel_test_1.mp4 frame 26
Processed apfel_test_1.mp4 frame 27
Processed apfel_test_1.mp4 frame 28
P

I0000 00:00:1754996779.312707 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1754996779.317644 60654990 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754996779.321502 60654997 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1754996779.325379 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1754996779.325650 60654391 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1754996779.328226 60655006 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754996779.335014 60655005 inference_feedback_manager.cc:114] Feedback man

Processed apfel_test_0.mp4 frame 1
Processed apfel_test_0.mp4 frame 2
Processed apfel_test_0.mp4 frame 3
Processed apfel_test_0.mp4 frame 4
Processed apfel_test_0.mp4 frame 5
Processed apfel_test_0.mp4 frame 6
Processed apfel_test_0.mp4 frame 7
Processed apfel_test_0.mp4 frame 8
Processed apfel_test_0.mp4 frame 9
Processed apfel_test_0.mp4 frame 10
Processed apfel_test_0.mp4 frame 11
Processed apfel_test_0.mp4 frame 12
Processed apfel_test_0.mp4 frame 13
Processed apfel_test_0.mp4 frame 14
Processed apfel_test_0.mp4 frame 15
Processed apfel_test_0.mp4 frame 16
Processed apfel_test_0.mp4 frame 17
Processed apfel_test_0.mp4 frame 18
Processed apfel_test_0.mp4 frame 19
Processed apfel_test_0.mp4 frame 20
Processed apfel_test_0.mp4 frame 21
Processed apfel_test_0.mp4 frame 22
Processed apfel_test_0.mp4 frame 23
Processed apfel_test_0.mp4 frame 24
Processed apfel_test_0.mp4 frame 25
Processed apfel_test_0.mp4 frame 26
Processed apfel_test_0.mp4 frame 27
Processed apfel_test_0.mp4 frame 28
P

OpenCV: Couldn't read video stream from file "../food/data/test/apfel/landmarks/apfel_test_1.mp4_landmarks.csv"
I0000 00:00:1754996780.190076 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1754996780.194986 60655033 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754996780.198928 60655037 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1754996780.203971 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1754996780.204259 60654391 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1754996780.206724 60655047 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp

...

OpenCV: Couldn't read video stream from file "../food/data/validation/erdbeere/landmarks/erdbeere_validation_1.mp4_landmarks.csv"
I0000 00:00:1754997070.622418 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1754997070.627159 60676079 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754997070.630860 60676079 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1754997070.634648 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1754997070.634929 60654391 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1754997070.637243 60676091 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inferen

Processed zwiebel_validation_1.mp4 frame 1
Processed zwiebel_validation_1.mp4 frame 2
Processed zwiebel_validation_1.mp4 frame 3
Processed zwiebel_validation_1.mp4 frame 4
Processed zwiebel_validation_1.mp4 frame 5
Processed zwiebel_validation_1.mp4 frame 6
Processed zwiebel_validation_1.mp4 frame 7
Processed zwiebel_validation_1.mp4 frame 8
Processed zwiebel_validation_1.mp4 frame 9
Processed zwiebel_validation_1.mp4 frame 10
Processed zwiebel_validation_1.mp4 frame 11
Processed zwiebel_validation_1.mp4 frame 12
Processed zwiebel_validation_1.mp4 frame 13
Processed zwiebel_validation_1.mp4 frame 14
Processed zwiebel_validation_1.mp4 frame 15
Processed zwiebel_validation_1.mp4 frame 16
Processed zwiebel_validation_1.mp4 frame 17
Processed zwiebel_validation_1.mp4 frame 18
Processed zwiebel_validation_1.mp4 frame 19
Processed zwiebel_validation_1.mp4 frame 20
Processed zwiebel_validation_1.mp4 frame 21
Processed zwiebel_validation_1.mp4 frame 22
Processed zwiebel_validation_1.mp4 frame 

I0000 00:00:1754997072.102137 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1754997072.107640 60676199 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754997072.111005 60676203 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1754997072.116206 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1754997072.116491 60654391 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1754997072.119163 60676214 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754997072.126561 60676214 inference_feedback_manager.cc:114] Feedback man

Processed zwiebel_validation_0.mp4 frame 10
Processed zwiebel_validation_0.mp4 frame 11
Processed zwiebel_validation_0.mp4 frame 12
Processed zwiebel_validation_0.mp4 frame 13
Processed zwiebel_validation_0.mp4 frame 14
Processed zwiebel_validation_0.mp4 frame 15
Processed zwiebel_validation_0.mp4 frame 16
Processed zwiebel_validation_0.mp4 frame 17
Processed zwiebel_validation_0.mp4 frame 18
Processed zwiebel_validation_0.mp4 frame 19
Processed zwiebel_validation_0.mp4 frame 20
Processed zwiebel_validation_0.mp4 frame 21
Processed zwiebel_validation_0.mp4 frame 22
Processed zwiebel_validation_0.mp4 frame 23
Processed zwiebel_validation_0.mp4 frame 24
Processed zwiebel_validation_0.mp4 frame 25
Processed zwiebel_validation_0.mp4 frame 26
Processed zwiebel_validation_0.mp4 frame 27
Processed zwiebel_validation_0.mp4 frame 28
Processed zwiebel_validation_0.mp4 frame 29
Processed zwiebel_validation_0.mp4 frame 30
Processed zwiebel_validation_0.mp4 frame 31
Processed zwiebel_validation_0.m

OpenCV: Couldn't read video stream from file "../food/data/validation/zwiebel/landmarks/zwiebel_validation_0.mp4_landmarks.csv"
I0000 00:00:1754997073.665931 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1754997073.671565 60676312 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754997073.675265 60676312 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1754997073.679782 60654391 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1754997073.680067 60654391 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1754997073.682552 60676324 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference